## Create The ONNX File

In [29]:
from special_neurons import get_most_negative_sets
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
# model_name = 'EleutherAI/gpt-neo-1.3B'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)
most_neg = get_most_negative_sets(model_og)
most_neg[0].prev_layer_name, most_neg[0].linear_layer_name

('gpt_neox.layers.0.mlp', 'gpt_neox.layers.0.mlp.dense_h_to_4h')

In [30]:
# Our max_norm is none
print(model_og.gpt_neox.embed_in.max_norm, model_og.gpt_neox.embed_in.norm_type, model_og.gpt_neox.embed_in.scale_grad_by_freq)

None 2.0 False


In [31]:
model_og

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [32]:
model_og.gpt_neox.layers[0].input_layernorm

LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [33]:
embd_matrix = model_og.gpt_neox.embed_in.weight
embd_matrix.shape

torch.Size([50304, 512])

In [34]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [35]:
inps_one_hot = get_input_after_tokenizer("hello world")
inps_one_hot.shape, inps_one_hot.nonzero()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


(torch.Size([2, 50304]),
 tensor([[    0, 25521],
         [    1,  1533]]))

In [36]:
N_TOKENS = inps_one_hot.shape[0]
attention_mask = torch.ones((1, N_TOKENS), dtype=torch.int)


class FixedAttentionMask(torch.nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.attn = model_og.gpt_neox.layers[0].attention

    def forward(self, x):
        # TODO: IDK IF THIS IS RIGHT for POSITION IDS or ATTENTION MASK
        return self.attn(x, attention_mask=attention_mask, position_ids=torch.arange(N_TOKENS).unsqueeze(0))


embed_linear = torch.nn.Linear(
    embd_matrix.shape[0], embd_matrix.shape[1], bias=False)
embed_linear.weight = torch.nn.Parameter(embd_matrix.T)


class SimplfiedLayerNorm(torch.nn.Module):
    def __init__(self, layernorm: torch.nn.LayerNorm) -> None:
        super().__init__()
        self.weight = layernorm.weight
        self.bias = layernorm.bias
        self.eps = layernorm.eps
        # TODO: not constant
        self.ones_linear = torch.nn.Linear(512, 512, bias=False)
        self.ones_linear.weight = torch.nn.Parameter(torch.ones((512, 512)))
        self.ones_linear_neg = torch.nn.Linear(512, 512, bias=False)
        self.ones_linear_neg.weight = torch.nn.Parameter(-1 * torch.ones((512, 512)))

    def forward(self, x):
        # TODO: I think that this can be made more efficient
        expectation_neg = self.ones_linear_neg(x)
        variance = self.ones_linear((((x + expectation_neg) ** 2)))
        x = (x + expectation_neg) / torch.sqrt(variance + self.eps)
        x = x * self.weight
        x = x + self.bias

        return x


class ModelSel(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.embed_linear = embed_linear
        self.layer_norm = SimplfiedLayerNorm(model_og.gpt_neox.layers[0].input_layernorm)
        self.attn = FixedAttentionMask()

    def forward(self, x):
        x = self.embed_linear(x)
        x = self.layer_norm(x)
        x = x.unsqueeze(0)
        x = self.attn(x)
        return x[0]

# G
# # TODO: add residuals?
# model_sel = torch.nn.Sequential(
#     # model_og.gpt_neox.embed_in,
#     embed_linear,
#     # model_og.gpt_neox.emb_dropout, # we have p = 0.0 and thus useless
#     model_og.gpt_neox.layers[0].input_layernorm,
#     FixedAttentionMask(),
#     # TODO: VERIFY THIS JAZZ
# )


model_sel = ModelSel()
model_sel

ModelSel(
  (embed_linear): Linear(in_features=50304, out_features=512, bias=False)
  (layer_norm): SimplfiedLayerNorm()
  (attn): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
  )
)

In [37]:
inps_one_hot.shape, embed_linear.weight.shape

(torch.Size([2, 50304]), torch.Size([512, 50304]))

In [38]:
inps_one_hot_formatted = inps_one_hot.float()

In [39]:
# embed_linear.forward(inps_one_hot).shape
model_sel(inps_one_hot_formatted)[0].shape

torch.Size([2, 512])

In [40]:
torch.onnx.export(model_sel, inps_one_hot_formatted,
                  'model_sel.onnx', verbose=False, opset_version=12)

/home/lev/.local/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:557: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len > self.max_seq_len_cached:
/home/lev/.local/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:257: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if key_length > self.bias.shape[-1]:
/home/lev/.local/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:282: TracerWarning: torch.tensor results are registered as constants in the trace. You can sa

In [41]:
import onnx

# Load the ONNX model
model = onnx.load("model_sel.onnx")

# Get the names of input and output nodes
input_names = [input.name for input in model.graph.input]
output_names = [output.name for output in model.graph.output]

print("Input Names:", input_names)
print("Output Names:", output_names)

In [ ]:
model.graph

## Use Mariboupy

In [55]:
# !export PYTHONPATH="$PYTHONPATH:/home/lev/code/research/softmax_bound/bounding-softmax/Marabou"
# !pip install onnx onnxruntime maraboupy --upgrade

In [56]:
import sys
import numpy as np

## %
# Path to Marabou folder if you did not export it

# TODO: this is a hack, fix it
# TODO: add Marabou installation details to readme
# sys.path.append('/home/lev/code/research/softmax_bound/bounding-softmax/Marabou')
# sys.path.append('/home/lev/code/research/softmax_bound/bounding-softmax/Marabou/maraboupy')
from maraboupy import Marabou

In [57]:
options = Marabou.createOptions(verbosity = 1)
print("Simple Attention Head")
filename = "model_sel.onnx"
network = Marabou.read_onnx(filename)#, inputNames=inputNames, outputNames=[outputName])

Simple Attention Head


## Play around with basic constraints